In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from models import HuBERTMultiHead
import pandas as pd

In [2]:
train_audio_folder = '/media/maximos/9C33-6BBD/data/melos_singers/Rebetika_vowels/train/Bellou/'
csv_path = '/media/maximos/9C33-6BBD/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)

In [4]:
task_labels_map = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_map[features_list[i]] = 1
# add singer identification
task_labels_map['singer_id'] = feats['singer_id'].max()+1 # accounting for zero

In [5]:
model = HuBERTMultiHead(task_labels_map=task_labels_map)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [7]:
print(training_data[0])

{'input_values': array([ 0.03695227,  0.00765143, -0.0463584 , ...,  0.0349304 ,
       -0.04109538,  0.        ], dtype=float32), 'labels': {'Unnamed: 0': 0, 'names': 'bellou_a_1', 'singer_id': 0, 'Pitch': 294.792755848602, 'SpectralCentroid': 1988.70548799565, 'SpectralSpread': 1236.07658212378, 'SpectralSkewness': 0.667606950418554, 'SpectralKurtosis': 2.23089609708891, 'SpectralFlatness': 6.13002798571896e-05, 'SpectralCrest': 0.166798680989045, 'SpectralSlope': 8.35676790222799e-08, 'SpectralDecrease': 0.0345231147320676, 'SpectralRollOff': 3619.18975289899, 'SpectralVariation': 0.633051436428841, 'SpectralFlux': 0.0021260109600433, 'HarmonicSpectralDeviation': 0.0050759970417123, 'Tristimulus_1': 0.0668331273198761, 'Tristimulus_2': 0.375872580535276, 'Tristimulus_3': 0.545649719091416, 'HarmonicOddToEvenRatio': 1.55359376773475, 'Inharmonicity': 0.096271184919293, 'HarmonicEnergy': 0.0136912304680307, 'NoiseEnergy': 0.0129354341375336, 'Noisiness': 0.464518212827659, 'HarmonicTo

In [8]:
dataloader = DataLoader(training_data, batch_size=8, shuffle=True, collate_fn=model.collate_fn)

In [9]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x709849affcb0>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 8, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x7097cf206f60>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x7097cf096690>, 'generator': None, 'collate_fn': <bound method HuBERTMultiHead.collate_fn of HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      

In [10]:
b = next(iter(dataloader))

entered collate_fn
input_values: [array([-0.00360867, -0.0270893 , -0.0493161 , ...,  0.13624442,
        0.11737493,  0.10249846], dtype=float32), array([-0.04315714, -0.07043701, -0.05468801, ..., -0.00907535,
       -0.01197426, -0.01709766], dtype=float32), array([-0.02886814, -0.01050005,  0.04926375, ...,  0.0175477 ,
       -0.08871207,  0.        ], dtype=float32), array([-0.08827513, -0.15996683, -0.00464054, ..., -0.15268797,
       -0.04472587,  0.        ], dtype=float32), array([ 0.03695227,  0.00765143, -0.0463584 , ...,  0.0349304 ,
       -0.04109538,  0.        ], dtype=float32), array([-0.13698706, -0.21405686, -0.22277153, ..., -0.026218  ,
       -0.02255014,  0.        ], dtype=float32), array([-0.05828063, -0.11266457, -0.11046638, ..., -0.03612334,
       -0.07856341,  0.        ], dtype=float32), array([0.05924646, 0.11763971, 0.11149202, ..., 0.08310056, 0.17159376,
       0.21682099], dtype=float32)]
labels: [{'Unnamed: 0': 0, 'names': 'bellou_a_1', 'singer_id

In [17]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))

8
8
8
